In [1]:
# Utility script

# bootstrap

In [2]:
import numpy as np

In [3]:
from util_registration import keypoint_estim
from util_statistic import outlier_remove
from util_registration import ncc_optimiz

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#%%  --- Perform multiple registrations (with bootstrap on ci [current image] samples) ---
def imreg__bs(ri, ci,
              nB,
              true_gt_tuple = None):
    '''
    Evaluate parameters estimates for geometric transformation with bootstrap.
    
    Input:
	    ri - reference image;
	    ci - current image;
	    nB - number of bootstrap runs;
	    true_gt_tuple - true transformation values for speed-up mode (used in this function): 
                                                       if 1st registration step estimates bad - we don't evaluate 2nd step
        								               (in real situation we don't know how bad or not 1st step estimates).
	
	Output:
	    par4_tuple - tuple of 4 estimates arrays (for center_x, center_y, scale, rotation) with nB length,
				center_x, center_y - ci center coordinates (relative to ri).

    Commets:
        ci must be squared.              # XXX - Attension.
   
    '''
    
    #%% 1 stage registration
    # Geometric transformation (gt) by keypoints   (no bootstrap).
    gt_kp = keypoint_estim(ri,ci)
    x0 = gt_kp
    
    
    # We calculate "NCC optimization" (2nd step) only if flag == True
    #    [flag == True if true_gt_tuple not specified,
    #                     or 1st step estimates are good (relative to true_gt_tuple)]
    flag = False
    if true_gt_tuple is None:
        flag = True
    else:
        # Check if the 1st step estimates are good.
        
        # Bounds on errors for speed-up mode (relative to true values). 
        sh_b = 4
        sc_b = 0.1
        rot_b = 10
        # Alternative: sh_b = 2, sc_b = 0.05, rot_b = 5 (as in Kybic_bs and functions.statistic.monteCarlo_reg)
        
        bound4_tuple = (sh_b, sh_b, sc_b, rot_b)
        bound4_array = np.array(bound4_tuple)
        
        true_gt_array = np.array(true_gt_tuple)
        gt_kp_array = np.array(gt_kp)
        
        true_abs_err4 = np.absolute( gt_kp_array - true_gt_array )
        if np.all(true_abs_err4 < bound4_array):
            flag = True
        
    
    
    #%%  
    ci_s = np.size(ci,0)
    nS = ci_s*ci_s
            
    xp_list = np.zeros( nB )
    yp_list = np.zeros( nB )
    sc_list = np.zeros( nB )
    rot_list = np.zeros( nB )
    if flag:
        A = np.random.randint(low=0, high=ci_s, size=(nS,2*nB))
        for iB in range(nB):
            x_list = A[:,iB] 
            y_list = A[:,nB+iB]
            
            # --- DEBUG ---
            # [Y_,X_] = np.meshgrid(np.arange(ci_s), np.arange(ci_s)) 
            # x_list = X_.flatten(order='F')
            # y_list = Y_.flatten(order='F')
            # -------------
            
            b_ind = np.ravel_multi_index((x_list,y_list), (ci_s,ci_s), order='F')
                        
            # if indices==None  ->  without bootstrap.
            gt_ncc = ncc_optimiz(ri,ci, x0,
                                 indices = b_ind)
            
            xp_list[iB] = gt_ncc[0]
            yp_list[iB] = gt_ncc[1]
            sc_list[iB] = gt_ncc[2]
            rot_list[iB] = gt_ncc[3]
            
        par4_tuple = xp_list, yp_list, sc_list, rot_list
            
    else:
        # We assign values so that all bootstrap estimates will considered outliers
        #    (bootstrap characteristics will be nan's).
        an_array =  np.arange(nB) 
        a_mean = np.mean(an_array)
        ind_low = an_array < a_mean
        ind_high = an_array >= a_mean
        an_array[ind_low] = -100
        an_array[ind_high] = 100
        par4_tuple = (an_array,)*4
    return par4_tuple



#%% --- Check bootstrap estimates ---
def bs__est_check(par4_tuple, bound4_tuple): 
    '''
    Remove bootstrap outliers.
    
    Input:
        par4_tuple - bootstrap estimates for 4 parameters (center_x, center_y, scale, rotation),
				center_x, center_y - ci center coordinates (relative to ri);
        bound4_tuple - bounds on errors.
        
    Output:
        good_par4__tuple - good bootstrap estimates for 4 parameters;
        indexes - indexes of good bootstrap estimates (ndarray);
        p - fraction of good estimates.
    '''

    
    #%%  
    (xp_list, yp_list, sc_list, rot_list) = par4_tuple
    err_x = xp_list - np.mean(xp_list)
    err_y = yp_list - np.mean(yp_list)
    err_sc = sc_list - np.mean(sc_list)
    err_rot = rot_list - np.mean(rot_list)
    par4_tuple = (err_x, err_y, err_sc, err_rot) 
    
    #%% Remove outliers.
    (good_par4__tuple,
     indexes)          = outlier_remove(par4_tuple,
                                        bound4_tuple)

    # Initial number of bootstrap samples.
    nB = xp_list.size

    p = indexes.size / nB

    return good_par4__tuple, indexes, p




#%%  
def Kybic_bs(ri, ci,
             nB,
             true_gt_tuple = None):
    '''
    Evaluate registration RMSE by bootstrap (Kybic, 2010).
      
    Input: 
        ri - reference image;
        ci - current image;
        nB - number of bootstrap iterations;
        true_gt_tuple - true transformation values for speed-up mode (used in this function): 
                                                    if 1st registration step estimates bad - we don't evaluate 2nd step;
        								            (in real situation we don't know how bad or not 1st step estimates)
        
    Output: 
        cov4_4 - covariance matrix for 4 parameters errors.
        
    Comments:                            
        We consider ci be squared.     # XXX - Attension.
    '''
    
    # Bounds on errors.  
    sh_b = 2
    sc_b = 0.05
    rot_b = 5
    
    bound4_tuple = (sh_b, sh_b, sc_b, rot_b)
    
    
    #%%
    bs_est4 = imreg__bs(ri, ci,
                        nB,
                        true_gt_tuple)
    good_par4__tuple, _, p = bs__est_check(bs_est4,bound4_tuple)
    X = np.stack(good_par4__tuple, axis=1)
    
    
    #%%
    # thresh = 0.9          # XXX - Attension.
    thresh = 0.00001
    if p >= thresh:   
        cov4_4 = np.cov(X, rowvar=False)
    else:
        cov4_4 = np.nan * np.ones( (4,4) )
    return cov4_4, p